In [1]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import string
import nltk
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr, kendalltau
import libvoikko

In [2]:
exclude = set(string.punctuation)

In [3]:
# !pip install libvoikko
# !pip install voikko

# Data Importing and Sorting

In [4]:
files = os.listdir('corpus')
files.remove('.DS_Store')
files.remove('scores_ru-en.csv')
scaler = MinMaxScaler()
for file_ in files:
  name = file_.split('-')[0] + file_.split('-')[1]
  vars()[name] = pd.read_csv(os.path.join('corpus', file_, 'scores.csv'))
  vars()[name].drop(columns = ['source', 'annotators', 'z-score'], inplace = True)
  vars()[name]['avg-score'] = scaler.fit_transform(vars()[name]['avg-score'].values.reshape(-1,1)) #normalizing values betwewen 0 and 1

In [5]:
english = csen.copy()
for df in [deen, ruen, zhen]:
  english = english.append(df)

In [6]:
finnish = enfi.copy()
chinese = enzh.copy()

In [7]:
english.reset_index(drop = True, inplace = True)

# Baseline assessment before preprocessment

### Finnish & Chinese

In [8]:
# Train, Validation & Test split
# en_train, en_dev = train_test_split(english, shuffle = True, test_size = 0.2, random_state = 7)
# en_dev, en_test = train_test_split(en_dev, shuffle = True, test_size = 0.5, random_state = 7)

fin_train, fin_dev = train_test_split(finnish, shuffle = True, test_size = 0.2, random_state = 7)
fin_dev, fin_test = train_test_split(fin_dev, shuffle = True, test_size = 0.5, random_state = 7)

ch_train, ch_dev = train_test_split(chinese, shuffle = True, test_size = 0.2, random_state = 7)
ch_dev, ch_test = train_test_split(ch_dev, shuffle = True, test_size = 0.5, random_state = 7)

In [9]:
# Transform dataset into a bag of words
baseline_encoder_reference = CountVectorizer(max_features = 5000)#settting limit for computational reasons
baseline_encoder_translation = CountVectorizer(max_features = 5000)
names = ['fin_train', 'fin_dev', 'fin_test', 'ch_train', 'ch_dev', 'ch_test']
for i,df in enumerate([fin_train, fin_dev, fin_test, ch_train, ch_dev, ch_test]):
    if (i == 0 or i == 3):
        vars()[names[i] + '_bl_encoded_reference'] = baseline_encoder_reference.fit_transform(df['reference']).todense()
        vars()[names[i] + '_bl_encoded_translation'] = baseline_encoder_translation.fit_transform(df['translation']).todense()
    else:
        vars()[names[i] + '_bl_encoded_reference'] = baseline_encoder_reference.transform(df['reference']).todense()
        vars()[names[i] + '_bl_encoded_translation'] = baseline_encoder_translation.transform(df['translation']).todense()
    vars()['y_' + names[i]] = np.array(df['avg-score'])

In [10]:
# Merge reference and translation
fin_train_bl_encoded = np.hstack((fin_train_bl_encoded_reference, fin_train_bl_encoded_translation))
fin_dev_bl_encoded = np.hstack((fin_dev_bl_encoded_reference, fin_dev_bl_encoded_translation))

ch_train_bl_encoded = np.hstack((ch_train_bl_encoded_reference, ch_train_bl_encoded_translation))
ch_dev_bl_encoded = np.hstack((ch_dev_bl_encoded_reference, ch_dev_bl_encoded_translation))

# baseline_regressor_fin = LinearRegression()
# baseline_regressor_ch = LinearRegression()

# baseline_regressor_fin.fit(fin_train_bl_encoded, y_fin_train)
# baseline_regressor_ch.fit(ch_train_bl_encoded, y_ch_train)

In [11]:
fin_train_bl_encoded_reference[0].shape

(1, 5000)

In [12]:
fin_train_bl_encoded_translation[0].shape

(1, 5000)

In [13]:
# FINNISH
cos_train_fin = []
for i in range(fin_train_bl_encoded_reference.shape[0]):
    cos_train_fin.append(cosine_similarity(fin_train_bl_encoded_reference[i], fin_train_bl_encoded_translation[i])[0])
cos_train_fin = np.array(cos_train_fin)
cos_train_fin.shape = (cos_train_fin.shape[0],)

cos_dev_fin = []
for i in range(fin_dev_bl_encoded_reference.shape[0]):
    cos_dev_fin.append(cosine_similarity(fin_dev_bl_encoded_reference[i], fin_dev_bl_encoded_translation[i])[0])
cos_dev_fin = np.array(cos_dev_fin)
cos_dev_fin.shape = (cos_dev_fin.shape[0],)

# CHINESE
cos_train_ch = []
for i in range(ch_train_bl_encoded_reference.shape[0]):
    cos_train_ch.append(cosine_similarity(ch_train_bl_encoded_reference[i], ch_train_bl_encoded_translation[i])[0])
cos_train_ch = np.array(cos_train_ch)
cos_train_ch.shape = (cos_train_ch.shape[0],)

cos_dev_ch = []
for i in range(ch_dev_bl_encoded_reference.shape[0]):
    cos_dev_ch.append(cosine_similarity(ch_dev_bl_encoded_reference[i], ch_dev_bl_encoded_translation[i])[0])
cos_dev_ch = np.array(cos_dev_ch)
cos_dev_ch.shape = (cos_dev_ch.shape[0],)

In [14]:
baseline_corr_train, baseline_corr_train_pvalue = pearsonr(y_fin_train, cos_train_fin)
baseline_corr_dev, baseline_corr_dev_pvalue = pearsonr(y_fin_dev, cos_dev_fin)

baseline_corr_ktau_train, baseline_corr_ktau_train_pvalue = kendalltau(y_fin_train, cos_train_fin)
baseline_corr_ktau_dev, baseline_corr_ktau_dev_pvalue = kendalltau(y_fin_dev, cos_dev_fin)

print(f'Finnish - Pearson correlation between cosine similarity and score on train set: {baseline_corr_train} (p-value < 0.001: {baseline_corr_train_pvalue < 0.001}); and Kendall Tau: {baseline_corr_ktau_train} (p-value < 0.001: {baseline_corr_ktau_train_pvalue < 0.001})')
print(f'Finnish - Pearson correlation between cosine similarity and score on development set: {baseline_corr_dev} (p-value < 0.001: {baseline_corr_dev_pvalue < 0.001}); and Kendall Tau: {baseline_corr_ktau_dev} (p-value < 0.001: {baseline_corr_ktau_dev_pvalue < 0.001})')

baseline_corr_train, baseline_corr_train_pvalue = pearsonr(y_ch_train, cos_train_ch)
baseline_corr_dev, baseline_corr_dev_pvalue = pearsonr(y_ch_dev, cos_dev_ch)

baseline_corr_ktau_train, baseline_corr_ktau_train_pvalue = kendalltau(y_ch_train, cos_train_ch)
baseline_corr_ktau_dev, baseline_corr_ktau_dev_pvalue = kendalltau(y_ch_dev, cos_dev_ch)

print(f'Chinese - Pearson correlation between cosine similarity and score on train set: {baseline_corr_train} (p-value < 0.001: {baseline_corr_train_pvalue < 0.001}); and Kendall Tau: {baseline_corr_ktau_train} (p-value < 0.001: {baseline_corr_ktau_train_pvalue < 0.001})')
print(f'Chinese - Pearson correlation between cosine similarity and score on development set: {baseline_corr_dev} (p-value < 0.001: {baseline_corr_dev_pvalue < 0.001}); and Kendall Tau: {baseline_corr_ktau_dev} (p-value < 0.001: {baseline_corr_ktau_dev_pvalue < 0.001})')

Finnish - Pearson correlation between cosine similarity and score on train set: -0.010170921922147021 (p-value < 0.001: False); and Kendall Tau: 0.010244488785508862 (p-value < 0.001: False)
Finnish - Pearson correlation between cosine similarity and score on development set: 0.046827973029132615 (p-value < 0.001: False); and Kendall Tau: 0.02773097227754495 (p-value < 0.001: False)
Chinese - Pearson correlation between cosine similarity and score on train set: 0.02730019389883727 (p-value < 0.001: False); and Kendall Tau: 0.02707228350345321 (p-value < 0.001: False)
Chinese - Pearson correlation between cosine similarity and score on development set: 0.020327258193715256 (p-value < 0.001: False); and Kendall Tau: 0.011426439858153755 (p-value < 0.001: False)


# Cleaning the corpus

In [ ]:
def clean(text_list,
          lower = False,
          keep_numbers = False,
          keep_expression = False,
          remove_char = False,
          remove_stop = False,
          remove_tag = False,
          lemmatize = False,
          stemmer = False,
          english = True
          ):
    """
    Function that a receives a list of strings and preprocesses it.
    
    :param text_list: List of strings.
    :param lemmatize: Tag to apply lemmatization if True.
    :param stemmer: Tag to apply the stemmer if True.
    """
    if english:
        lang = 'english'
    else:
        lang = 'finnish'
    
    stop = set(stopwords.words(lang))
    stem = SnowballStemmer(lang)
    
    updates = []
    for j in tqdm(range(len(text_list))):
        
        text = text_list[j]
        
        #LOWERCASE TEXT
        if lower:
            text = text.lower()
            
        #KEEP NUMBERS AS TOKENS
        if keep_numbers:
            text = re.sub("[\d+]", 'NUMBER', text)
        
        #KEEP '?' and '!' AS TOKENS
        if keep_expression:
            text = re.sub("[\?|\!]", 'EXPRESSION', text)
            
        #REMOVE THAT IS NOT TEXT
        if remove_char:
            text = re.sub("[^a-zA-Z]", ' ', text)
            
        #REMOVE TAGS
        if remove_tag:
            text = BeautifulSoup(text).get_text()
        
        #REMOVE STOP WORDS
        if remove_stop:
            text = ' '.join([word for word in text.split(' ') if word not in stop])
        
        #LEMMATIZATION
        if lemmatize:
            if english:
                lemma = WordNetLemmatizer()
                text = " ".join(lemma.lemmatize(word) for word in text.split())
#             else:
#                 lemma = libvoikko.Voikko(u"fi")
#                 text = " ".join(lemma.analyze(word)[0]['BASEFORM'] for word in text.split())
        
        #STEMMER
        if stemmer:
            text = " ".join(stem.stem(word) for word in text.split())
        
        updates.append(text)
        
    return updates

def clean_zh_stopwords(text_list, keep_numbers=False, remove_punctuation=False, remove_stop = False, stopwords_set='merged'):
    """
    Function that removes chinese stopwords
    
    :param stopwords_set: remove words of both sets (merged), just the 1st (fst) or just the second (snd) 
    """
    updates = []
    
    zh_stopwords1 = [line.strip() for line in open('chinese_stopwords/chinese_stopwords1.txt', 'r', encoding='utf-8').readlines()]
    zh_stopwords2 = [line.strip() for line in open('chinese_stopwords/chinese_stopwords2.txt', 'r', encoding='utf-8').readlines()]
    
    if stopwords_set == 'merged':
        stop = list(set(zh_stopwords1 + zh_stopwords2))
    elif stopwords_set == 'fst':
        stop = zh_stopwords1
    elif stopwords_set == 'snd':
        stop = zh_stopwords2
        

    for j in tqdm(range(len(text_list))):
        
        text = text_list[j]
        
        #KEEP NUMBERS AS TOKENS
        if keep_numbers:
            text = re.sub("[\d+]", 'NUMBER', text)
        
        # REMOVE PUNCTUATION
        if remove_punctuation:
            # https://stackoverflow.com/questions/36640587/how-to-remove-chinese-punctuation-in-python
            punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
            text = re.sub(r"[%s]+" %punc, "", text)
            
        # REMOVE STOP WORDS
        if remove_stop:
            text = ' '.join([word for word in text.split(' ') if word not in stop])
            updates.append(text)
        
    return updates
        
    
def update_df(dataframe, list_updated):
    dataframe.update(pd.DataFrame({"Text": list_updated}))

In [ ]:
# english_cleaned = pd.DataFrame()
# english_cleaned['avg-score'] = english['avg-score']
# for column in ['reference', 'translation']:
#     english_cleaned[column] = clean(english[column], lower = True, 
#                                                     remove_char = True,
#                                                     remove_stop = True,
#                                                     lemmatize = True,
#                                                     stemmer = False)

In [ ]:
# english_cleaned.head()

In [ ]:
finnish_cleaned = pd.DataFrame()
finnish_cleaned['avg-score'] = finnish['avg-score']
for column in ['reference', 'translation']:
    finnish_cleaned[column] = clean(finnish[column], lower = True, 
                                                    remove_char = True,
                                                    lemmatize = True,
                                                    stemmer = False,
                                                    english=False)
finnish_cleaned.head()

In [ ]:
chinese_cleaned = pd.DataFrame()
chinese_cleaned['avg-score'] = chinese['avg-score']
for column in ['reference', 'translation']:
    chinese_cleaned[column] = clean_zh_stopwords(chinese[column], remove_punctuation=True, remove_stop = True)
chinese_cleaned.head()

# Train, Dev & Test Split

In [ ]:
# en_train, en_dev = train_test_split(english_cleaned, shuffle = True, test_size = 0.2, random_state = 7)
# en_dev, en_test = train_test_split(en_dev, shuffle = True, test_size = 0.5, random_state = 7)

fin_train, fin_dev = train_test_split(finnish_cleaned, shuffle = True, test_size = 0.2, random_state = 7)
fin_dev, fin_test = train_test_split(fin_dev, shuffle = True, test_size = 0.5, random_state = 7)

ch_train, ch_dev = train_test_split(chinese_cleaned, shuffle = True, test_size = 0.2, random_state = 7)
ch_dev, ch_test = train_test_split(ch_dev, shuffle = True, test_size = 0.5, random_state = 7)

# Baseline model assessment 

In [ ]:
baseline_encoder = CountVectorizer(max_features = 5000)#settting limit for computational reasons
names = ['fin_train', 'fin_dev', 'fin_test']
for i,df in enumerate([fin_train, fin_dev, fin_test]):
    for column in ['reference', 'translation']:
        encoded_df = names[i] + '_bl_encoded_' + column
        if i == 0:
            vars()[encoded_df] = baseline_encoder.fit_transform(df[column]).todense()
        else:
            vars()[encoded_df] = baseline_encoder.transform(df[column]).todense()
            
    y_name = 'y_' + names[i].split('_')[1]
    vars()[y_name] = np.array(df['avg-score'])

In [ ]:
fin_train_bl_encoded_reference.shape

# Encoding (for now only bag of words)

In [ ]:
encoder = CountVectorizer()
names = ['en_train', 'en_dev', 'en_test']
for i,df in enumerate([en_train, en_dev, en_test]):
    for column in ['reference', 'translation']:
        encoded_df = names[i] + '_encoded_' + column
        if i == 0:
            vars()[encoded_df] = encoder.fit_transform(df[column]).todense()
        else:
            vars()[encoded_df] = encoder.transform(df[column]).todense()
            
    y_name = 'y_' + names[i].split('_')[1]
    vars()[y_name] = np.array(df['avg-score'])

# Distance

In [ ]:
cos_dev = []
for i in range(en_dev_encoded_reference.shape[0]):
    cos_dev.append(cosine_similarity(e_dev_encoded_reference[i], en_dev_encoded_translation[i])[0])
cos_dev = np.array(cos_dev)
cos_dev.shape = (cos_dev.shape[0],)

cos_test = []
for i in range(en_test_encoded_reference.shape[0]):
    cos_test.append(cosine_similarity(en_test_encoded_reference[i], en_test_encoded_translation[i])[0])
cos_test = np.array(cos_test)
cos_test.shape = (cos_test.shape[0],)

In [ ]:
cleaned_corr_dev, cleaned_corr_dev_pvalue = pearsonr(y_dev, cos_dev)
cleaned_corr_test, cleaned_corr_test_pvalue = pearsonr(y_test,cos_test)

cleaned_corr_ktau_dev, cleaned_corr_ktau_dev_pvalue = kendalltau(y_dev, cos_dev)
cleaned_corr_ktau_test, cleaned_corr_ktau_test_pvalue = kendalltau(y_test,cos_test)

print(f'Pearson correlation between cosine similarity and score on development set: {cleaned_corr_dev} (p-value < 0.001: {cleaned_corr_dev_pvalue < 0.001}); and Kendall Tau: {cleaned_corr_ktau_dev} (p-value < 0.001: {cleaned_corr_ktau_dev_pvalue < 0.001})')
print(f'Pearson correlation between cosine similarity and score on test set: {cleaned_corr_test} (p-value < 0.001: {cleaned_corr_test_pvalue < 0.001}); and Kendall Tau: {cleaned_corr_ktau_test} (p-value < 0.001: {cleaned_corr_ktau_test_pvalue < 0.001})')